<a href="https://colab.research.google.com/github/mirklys/little-projects/blob/main/thesis/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install Box2D
!pip3 install box2d-py
!pip3 install gym[all]
!pip3 install gym[Box_2D]

In [ ]:
!pip install stable_baselines
!pip install stable_baselines3

     |████████████████████████████████| 240 kB 5.3 MB/s 
     |████████████████████████████████| 177 kB 5.2 MB/s 
     |████████████████████████████████| 1.5 MB 42.8 MB/s 
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616825 sha256=af06100b1b346cd9dc2bce2bda2c107cfbca17d7e59baf15a2710c39e9004121
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
Successfully built gym


In [ ]:
!pip install tensorflow

In [ ]:
import gym
import os
import numpy as np
import torch as th
from torch import nn
from torch.distributions.bernoulli import Bernoulli
import matplotlib.pyplot as plt
from collections import defaultdict
import time

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.utils import get_device
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines.common import set_global_seeds, make_vec_env
from stable_baselines3.common.monitor import Monitor

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
PATH_BASE = '/content/gdrive/MyDrive/Thesis Project'
PATH_DATA = os.path.join(PATH_BASE, 'data/')
PATH_NETWORKS = os.path.join(PATH_BASE, 'networks/')
PATH_PLOTS = os.path.join(PATH_BASE, 'plots/')
PATH_RESULTS = os.path.join(PATH_BASE, 'results/')
PATH_LOGS = os.path.join(PATH_BASE, 'logs/')
os.chdir(PATH_BASE)

In [ ]:
def test_network(game:str, size:int, dropout:float) -> str:
    title = "{}.{}x{}.dropout_{}".format(game, size, size, dropout)
    env = make_vec_env(game, n_envs=10, seed=0, vec_env_cls=DummyVecEnv)
    model = PPO.load(os.path.join(PATH_NETWORKS, game, title))

    rew, std = evaluate_policy(model, env, n_eval_episodes=10)

    return "Networks cumulative reward {:.2f} ±{:.2f}".format(rew, std)

In [ ]:
test_network('BipedalWalker-v3', 128, 0.0)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


'Networks cumulative reward 317.78 ±0.60'

In [ ]:
test_network('BipedalWalker-v3', 128, 0.2)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


'Networks cumulative reward 200.50 ±135.86'

In [ ]:
def parse_title(title: str) -> tuple:
    splitted = title.split(".")
    game = splitted[0]
    size = splitted[1].split("x")[0]
    dropout = float(title.split("_")[1])
    return game, size, dropout

In [ ]:
def eval_with_mask(model, percent_to_mask, size, env):
    rewards = []
    for i in range(20): # repeat over multiple random masks)
        model.policy.features_extractor.mask_units(percent_to_mask=percent_to_mask, size=size)
        mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
        rewards.append(mean_reward)
    return np.round(np.mean(rewards), 3), np.round(np.std(rewards), 3)

In [ ]:
import time
import warnings

num_training_steps = 5000 # train for at least 100-150k steps
warnings.filterwarnings("ignore")

nets = os.listdir(os.path.join(PATH_NETWORKS, 'CartPole-v1'))
evals = defaultdict(list)
env = make_vec_env('CartPole-v1', n_envs=10, seed=0, vec_env_cls=DummyVecEnv)
for net in nets:
    game, size, dropout = parse_title(net)
    model = PPO.load(os.path.join(PATH_NETWORKS, game, net))
    model.set_env(env)
    model.policy.features_extractor.training = False
    print("Masking size: ")
    mean_rews, std_rews = [], []
    for mask in np.arange(0.0, 1.05, .05):
        print(mask, end=" ")
        eval = eval_with_mask(model, percent_to_mask=float(mask), size=model.policy.features_extractor.l2_size, env=env)
        print(eval, end='\n')
        mean_rews.append(eval[0])
        std_rews.append(eval[1])
    print('\n')
    evals[(game, size, dropout)].append((mean_rews, std_rews))